In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [142]:
afreq_f <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/afreq_20201012/ukb24983_cal_hla_cnv.afreq_20201012.pvar.zst'
old_cal_annot_f <- '/oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.20200701.tsv.gz'
hardy_f   <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/afreq_20201012/plink_output/ukb24983_cal_hla_cnv.white_british.hardy.zst'
hardy_x_f <- str_replace(hardy_f, '.zst$', '.x.zst')

hla_pvar_f <- '/oak/stanford/groups/mrivas/ukbb24983/hla/pgen/ukb_hla_v3.pvar.zst'
cnv_pvar_f <- '/oak/stanford/groups/mrivas/ukbb24983/cnv/pgen/cnv.pvar'

# output
var_QC_plot_f <- 'variant_QC.pdf'
var_QC_f <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/annotation_20201012/ukb24983_cal_hla_cnv.var_QC.tsv'


## Read the data files

In [4]:
cat_or_zcat <- function(f){
    ifelse(endsWith(f, '.zst'), 'zstdcat', ifelse(endsWith(f, '.gz'), 'zcat', 'cat'))
}

fread_CHROM <- function(f){
    fread(cmd=paste(cat_or_zcat(f), f), colClasses = c('#CHROM'='character')) %>% rename('CHROM'='#CHROM')
}


In [69]:
afreq_f %>% fread_CHROM() %>% left_join(
    bind_rows(
        hardy_f   %>% fread_CHROM() %>% select(ID, MIDP),
        hardy_x_f %>% fread_CHROM() %>% select(ID, MIDP)
    ) %>% rename('hwe_p'='MIDP'), by='ID'
) %>% mutate(
    # HWE filter, pass if hwe p >= 1e-7
    filter_hwe = if_else(is.na(hwe_p) | (hwe_p >= 1e-7), '', 'hwe;'),
    
    # Missigness filter, pass if missigness <= 1%
    filter_missingness = 0.01 >= if_else(
        is.na(array) | array == 'both', f_miss,
        if_else(array == 'UKBB', f_miss_UKBB, f_miss_UKBL)
    ),
    filter_missingness = if_else(filter_missingness, '', 'missingness;')
) -> afreq_hwe_df


In [11]:
old_cal_annot_f %>% fread_CHROM() -> old_cal_annot_df


In [127]:
hla_pvar_f %>% fread_CHROM() -> hla_pvar_df
cnv_pvar_f %>% fread_CHROM() -> cnv_pvar_df


## check the previous filtering criteria

- `missingness`: QC filter - missingness
- `hwe`: QC filter - HWE p-value
- `mcpi`: QC filter: manual cluster plot inspection
- `gnomad_af`: QC filter - maf comparison with gnomAD
- `mgi`: QC filter - manual ??
- `mgi_notes`: QC filter - notes
- `all_filters`: QC filter summary


In [59]:
old_cal_annot_df %>%
mutate(
    FILTER = str_replace(paste0(
        if_else(is.na(missingness) | missingness == 0, '', 'missingness;'),
        if_else(is.na(hwe) | hwe == 0, '', 'hwe;'),
        if_else(is.na(mcpi) | mcpi == 0, '', 'mcpi;'),
        if_else(is.na(gnomad_af) | gnomad_af %in% c('', 'PASS'), '', 'gnomad_af;'),
        if_else(is.na(mgi) | mgi %in% c('', 'PASS', 'NOT_PTV'), '', 'mgi;')
    ), ';$', '')
) %>%
count(FILTER, filter, all_filters, missingness, hwe, mcpi, gnomad_af, mgi)


FILTER,filter,all_filters,missingness,hwe,mcpi,gnomad_af,mgi,n
<chr>,<lgl>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>
,NA,0,0,0,0,,,651983
,NA,0,0,0,0,,NOT_PTV,29
,NA,0,0,0,0,,PASS,53
,NA,0,0,0,0,PASS,,3589
,NA,NA,NA,NA,NA,,,19813
gnomad_af,NA,1,0,0,0,FAIL,,166
hwe,NA,1,0,1,0,,,34200
hwe,NA,1,0,1,0,,NOT_PTV,3
hwe,NA,1,0,1,0,,PASS,1


## compare the new QC flag and the previous ones

In [70]:
afreq_hwe_df %>%
left_join(old_cal_annot_df %>% select(ID, hwe), by='ID') %>%
count(filter_hwe, hwe)

filter_hwe,hwe,n
<chr>,<int>,<int>
,0,727645
,1,47
,NA,294586
hwe;,0,35
hwe;,1,56529
hwe;,NA,2126


In [71]:
afreq_hwe_df %>%
left_join(old_cal_annot_df %>% select(ID, missingness), by='ID') %>%
count(filter_missingness, missingness)

filter_missingness,missingness,n
<chr>,<int>,<int>
,0,685526
,1,422
,NA,18539
missingness;,0,4614
missingness;,1,93694
missingness;,NA,278173


In [125]:
old_cal_annot_df %>% 
count(all_filters, ld_indep)

all_filters,ld_indep,n
<int>,<lgl>,<int>
0,FALSE,294230
0,TRUE,361424
1,FALSE,106246
1,TRUE,1
2,FALSE,22346
3,FALSE,9
NA,NA,19813


## assigne variant QC flags

In [129]:
afreq_hwe_df %>%
left_join(old_cal_annot_df %>% select(ID, mcpi, gnomad_af, mgi, mgi_notes), by='ID') %>%
replace_na(list(mgi_notes='')) %>% mutate(
    FILTER = str_replace(paste0(
        filter_missingness, filter_hwe,
        if_else(is.na(mcpi) | mcpi == 0, '', 'mcpi;'),
        if_else(is.na(gnomad_af) | gnomad_af %in% c('', 'PASS'), '', 'gnomad_af;'),
        if_else(is.na(mgi) | mgi %in% c('', 'PASS', 'NOT_PTV'), '', 'mgi;')
    ), ';$', ''),
    FILTER = if_else(FILTER=='', '.', FILTER)
) %>%
select(-mcpi, -gnomad_af, -mgi, -filter_missingness, -filter_hwe) %>%
mutate(
    # compute minor allele frequency
    UKB_white_british_MAF = pmin(UKB_white_british_AF, 1 - UKB_white_british_AF),
    
    # which genotype data source contains the variant (array [cal], CNV [cnv], or HLA [hla])
    geno_data_source = if_else(ID %in% (cnv_pvar_df %>% pull(ID)), 'cnv', if_else(ID %in% (hla_pvar_df %>% pull(ID)), 'hla', 'cal'))
)-> afreq_hwe_qc_df


In [135]:
afreq_hwe_qc_df %>% count(geno_data_source, array)

geno_data_source,array,n
<chr>,<chr>,<int>
cal,both,753693
cal,UKBB,34197
cal,UKBL,17536
cnv,NA,275180
hla,NA,362


## plot the Venn Diagram 

In [115]:
filter_key <- c('missingness', 'hwe', 'mcpi', 'gnomad_af', 'mgi') 

var_QC_p <- UpSetR::upset(
    UpSetR::fromList(setNames(filter_key %>% lapply(function(k){afreq_hwe_qc_df %>% filter(str_detect(FILTER, k)) %>% pull(ID)}), filter_key)),
    mainbar.y.label = "Number of removed variants",
    sets.x.label = "# variants", nsets = 20, nintersects = NA,
    text.scale = 1.3, order.by = "freq", show.numbers = "yes"
)


In [124]:
pdf(file=var_QC_plot_f, onefile=FALSE, height = 6, width=8, family = "Helvetica")
var_QC_p
dev.off()


png 
  2

In [118]:
afreq_hwe_qc_df %>% count(FILTER) %>% arrange(-n)


FILTER,n
<chr>,<int>
.,669598
missingness,352422
hwe,34668
missingness;hwe,24008
gnomad_af,164
mgi,41
missingness;gnomad_af,40
mcpi,10
missingness;hwe;gnomad_af,6


## save the results to a file

In [138]:
cols <- c('CHROM', 'POS', 'ID', 'REF', 'ALT', 'FILTER', 'geno_data_source', 'array', 'UKB_white_british_MAF', 'hwe_p', 'mgi_notes', 'f_miss', 'f_miss_UKBB', 'f_miss_UKBL')


In [143]:
afreq_hwe_qc_df %>%
select(all_of(c(cols, setdiff(colnames(afreq_hwe_qc_df), cols)))) %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(var_QC_f, sep='\t', na = "NA", quote=F)
